# National Drug Code Dataset

In [1]:
# Dependencies

# Transformation
import pandas as pd
import numpy as np
import io

# Excel writer
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

In [2]:
# Import CSVs
product_df = pd.read_csv("./input/NDCproduct.csv", encoding='latin-1', dtype={'STARTMARKETINGDATE': 'str',
                                                                                     'ENDMARKETINGDATE': 'str',
                                                                                    'LISTING_RECORD_CERTIFIED_THROUGH' : 'str'})
package_df = pd.read_csv("./input/NDCpackage.csv", encoding='latin-1')

# Merge dataframes
ndc_df = product_df.merge(package_df, on=["PRODUCTID"])

# Preview
ndc_df.head()

,PRODUCTID,PRODUCTNDC_x,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE_x,ENDMARKETINGDATE_x,...,DEASCHEDULE,NDC_EXCLUDE_FLAG_x,LISTING_RECORD_CERTIFIED_THROUGH,PRODUCTNDC_y,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE_y,ENDMARKETINGDATE_y,NDC_EXCLUDE_FLAG_y,SAMPLE_PACKAGE
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,...,NaN,N,20191231,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710.0,NaN,N,N
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,...,NaN,N,20191231,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601.0,NaN,N,N
2,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,...,NaN,N,20191231,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601.0,NaN,N,N
3,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,HUMAN PRESCRIPTION DRUG,Quinidine Gluconate,NaN,Quinidine Gluconate,SOLUTION,INTRAVENOUS,19500712,NaN,...,NaN,N,20191231,0002-1407,0002-1407-01,10 mL in 1 VIAL (0002-1407-01),19510301.0,NaN,N,N
4,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,...,NaN,N,20201231,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107.0,NaN,N,Y


In [3]:
ndc_df.columns.tolist()

['PRODUCTID',
 'PRODUCTNDC_x',
 'PRODUCTTYPENAME',
 'PROPRIETARYNAME',
 'PROPRIETARYNAMESUFFIX',
 'NONPROPRIETARYNAME',
 'DOSAGEFORMNAME',
 'ROUTENAME',
 'STARTMARKETINGDATE_x',
 'ENDMARKETINGDATE_x',
 'MARKETINGCATEGORYNAME',
 'APPLICATIONNUMBER',
 'LABELERNAME',
 'SUBSTANCENAME',
 'ACTIVE_NUMERATOR_STRENGTH',
 'ACTIVE_INGRED_UNIT',
 'PHARM_CLASSES',
 'DEASCHEDULE',
 'NDC_EXCLUDE_FLAG_x',
 'LISTING_RECORD_CERTIFIED_THROUGH',
 'PRODUCTNDC_y',
 'NDCPACKAGECODE',
 'PACKAGEDESCRIPTION',
 'STARTMARKETINGDATE_y',
 'ENDMARKETINGDATE_y',
 'NDC_EXCLUDE_FLAG_y',
 'SAMPLE_PACKAGE']

## Cleaning columns

In [4]:
# Drop duplicate columns
ndc_df = ndc_df.drop(columns = ["PRODUCTNDC_y", 'STARTMARKETINGDATE_y','ENDMARKETINGDATE_y', 'NDC_EXCLUDE_FLAG_y'])

# Rename columns to be more descriptive
ndc_df = ndc_df.rename(columns= {
    'PRODUCTNDC_x' : 'PRODUCT_NDC',
    'STARTMARKETINGDATE_x' : 'MARKETING_STARTDATE',
    'ENDMARKETINGDATE_x' : 'MARKETING_ENDDATE',
    'NDC_EXCLUDE_FLAG_x' : 'NDC_EXCLUDE_FLAG'
})

In [5]:
# Convert key columns to lowercase
ndc_df[["PRODUCTTYPENAME", "PROPRIETARYNAME", "NONPROPRIETARYNAME", "DOSAGEFORMNAME", "ROUTENAME", "MARKETINGCATEGORYNAME", "LABELERNAME", "SUBSTANCENAME"]] = \
ndc_df[["PRODUCTTYPENAME", "PROPRIETARYNAME", "NONPROPRIETARYNAME", "DOSAGEFORMNAME", "ROUTENAME", "MARKETINGCATEGORYNAME", "LABELERNAME", "SUBSTANCENAME"]]\
.apply(lambda x: x.astype(str).str.lower())

In [6]:
ndc_df.head()

,PRODUCTID,PRODUCT_NDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETING_STARTDATE,MARKETING_ENDDATE,...,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,NDCPACKAGECODE,PACKAGEDESCRIPTION,SAMPLE_PACKAGE
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,human otc drug,sterile diluent,NaN,diluent,"injection, solution",subcutaneous,19870710,NaN,...,water,1,mL/mL,NaN,NaN,N,20191231,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,N
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,20120601,NaN,...,florbetapir f-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",N
2,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,20120601,NaN,...,florbetapir f-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",N
3,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,human prescription drug,quinidine gluconate,NaN,quinidine gluconate,solution,intravenous,19500712,NaN,...,quinidine gluconate,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,20191231,0002-1407-01,10 mL in 1 VIAL (0002-1407-01),N
4,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,human prescription drug,trulicity,NaN,dulaglutide,"injection, solution",subcutaneous,20140918,NaN,...,dulaglutide,0.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,Y


In [63]:
## Cleaning duplicates/typos in LABELERNAME
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('walmart'), 'LABELERNAME'] = 'walmart'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('wal-mart'), 'LABELERNAME'] = 'walmart'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('preferred'), 'LABELERNAME'] = 'preferred pharmaceuticals'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('preferreed'), 'LABELERNAME'] = 'preferred pharmaceuticals'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('cvs'), 'LABELERNAME'] = 'cvs'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('walgreen'), 'LABELERNAME'] = 'walgreen'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('johnson &'), 'LABELERNAME'] = 'johnson & johnson'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('rite aid'), 'LABELERNAME'] = 'rite aid'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('rite-aid'), 'LABELERNAME'] = 'rite aid'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('target'), 'LABELERNAME'] = 'target'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('accord healthcare'), 'LABELERNAME'] = 'accord healthcare'
ndc_df.loc[ndc_df['LABELERNAME'].str.contains('acme'), 'LABELERNAME'] = 'acme united'

## Cleaning Date Fields

In [64]:
ndc_df['MARKETING_STARTDATE'] = pd.to_datetime(ndc_df['MARKETING_STARTDATE'], format='%Y%m%d', errors='coerce')
ndc_df['MARKETING_ENDDATE'] = pd.to_datetime(ndc_df['MARKETING_ENDDATE'], format='%Y%m%d', errors='coerce')
ndc_df['LISTING_RECORD_CERTIFIED_THROUGH'] = pd.to_datetime(ndc_df['LISTING_RECORD_CERTIFIED_THROUGH'], format='%Y%m%d', errors='coerce')
ndc_df.head()

,PRODUCTID,PRODUCT_NDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETING_STARTDATE,MARKETING_ENDDATE,...,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,NDCPACKAGECODE,PACKAGEDESCRIPTION,SAMPLE_PACKAGE,Marketing Start Year
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,human otc drug,sterile diluent,NaN,diluent,"injection, solution",subcutaneous,1987-07-10,NaT,...,1,mL/mL,NaN,NaN,N,2019-12-31,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,N,1987
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,2012-06-01,NaT,...,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,2019-12-31,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",N,2012
2,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,2012-06-01,NaT,...,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,2019-12-31,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",N,2012
3,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,human prescription drug,quinidine gluconate,NaN,quinidine gluconate,solution,intravenous,1950-07-12,NaT,...,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,2019-12-31,0002-1407-01,10 mL in 1 VIAL (0002-1407-01),N,1950
4,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,human prescription drug,trulicity,NaN,dulaglutide,"injection, solution",subcutaneous,2014-09-18,NaT,...,0.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,2020-12-31,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,Y,2014


In [65]:
# Create column for Marketing Start Year
ndc_df["Marketing Start Year"] = ndc_df["MARKETING_STARTDATE"].astype(str).str[0:4]
ndc_df.head()

,PRODUCTID,PRODUCT_NDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETING_STARTDATE,MARKETING_ENDDATE,...,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,NDCPACKAGECODE,PACKAGEDESCRIPTION,SAMPLE_PACKAGE,Marketing Start Year
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,human otc drug,sterile diluent,NaN,diluent,"injection, solution",subcutaneous,1987-07-10,NaT,...,1,mL/mL,NaN,NaN,N,2019-12-31,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,N,1987
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,2012-06-01,NaT,...,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,2019-12-31,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",N,2012
2,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,2012-06-01,NaT,...,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,2019-12-31,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",N,2012
3,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,human prescription drug,quinidine gluconate,NaN,quinidine gluconate,solution,intravenous,1950-07-12,NaT,...,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,2019-12-31,0002-1407-01,10 mL in 1 VIAL (0002-1407-01),N,1950
4,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,human prescription drug,trulicity,NaN,dulaglutide,"injection, solution",subcutaneous,2014-09-18,NaT,...,0.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,2020-12-31,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,Y,2014


In [66]:
ndc_df.drop_duplicates(inplace= True)
ndc_df.head()

,PRODUCTID,PRODUCT_NDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,MARKETING_STARTDATE,MARKETING_ENDDATE,...,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH,NDCPACKAGECODE,PACKAGEDESCRIPTION,SAMPLE_PACKAGE,Marketing Start Year
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,human otc drug,sterile diluent,NaN,diluent,"injection, solution",subcutaneous,1987-07-10,NaT,...,1,mL/mL,NaN,NaN,N,2019-12-31,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,N,1987
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,2012-06-01,NaT,...,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,2019-12-31,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",N,2012
2,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,human prescription drug,amyvid,NaN,florbetapir f 18,"injection, solution",intravenous,2012-06-01,NaT,...,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,2019-12-31,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",N,2012
3,0002-1407_14757f9d-f641-4836-acf3-229265588d1d,0002-1407,human prescription drug,quinidine gluconate,NaN,quinidine gluconate,solution,intravenous,1950-07-12,NaT,...,80,mg/mL,"Antiarrhythmic [EPC],Cytochrome P450 2D6 Inhib...",NaN,N,2019-12-31,0002-1407-01,10 mL in 1 VIAL (0002-1407-01),N,1950
4,0002-1433_4468578a-47d2-488e-9fd4-a8322070392f,0002-1433,human prescription drug,trulicity,NaN,dulaglutide,"injection, solution",subcutaneous,2014-09-18,NaT,...,0.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,2020-12-31,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,Y,2014


## Create Summary DataFrames

### re-do route/dosage - count if contains ...
### add info to coverpage

In [67]:
# Count of Drugs by Proprietary Name
prop_df = ndc_df[["PROPRIETARYNAME", "NDCPACKAGECODE"]].groupby(["PROPRIETARYNAME"]).count()\
.reset_index()\
.rename(columns= {"NDCPACKAGECODE" : "Count of Drugs", "PROPRIETARYNAME" : "Proprietary Name"})\
.sort_values(by= "Count of Drugs", ascending=False)

prop_df.head()

,Proprietary Name,Count of Drugs
28339,oxygen,5729
19572,ibuprofen,2478
16291,gabapentin,1706
24568,metformin hydrochloride,1337
26808,nitrogen,1327


In [68]:
drugtype_df = ndc_df[["PRODUCTTYPENAME", "NDCPACKAGECODE"]].groupby(["PRODUCTTYPENAME"]).count()\
.reset_index()\
.rename(columns= {"NDCPACKAGECODE" : "Count of Drugs", "PRODUCTTYPENAME" : "Drug Type"})\
.sort_values(by= "Count of Drugs", ascending=False)\

drugtype_df.head()

,Drug Type,Count of Drugs
2,human prescription drug,132341
1,human otc drug,100936
3,non-standardized allergenic,4531
4,plasma derivative,433
5,standardized allergenic,326


In [69]:
dosage_df = ndc_df[["DOSAGEFORMNAME", "NDCPACKAGECODE"]].groupby(["DOSAGEFORMNAME"]).count()\
.reset_index()\
.rename(columns= {"NDCPACKAGECODE" : "Count of Drugs", "DOSAGEFORMNAME" : "Dosage Form"})\
.sort_values(by= "Count of Drugs", ascending=False)\

dosage_df.head()

,Dosage Form,Count of Drugs
116,tablet,49395
123,"tablet, film coated",25780
56,"injection, solution",20249
69,liquid,19039
6,capsule,13172


In [70]:
route_df = ndc_df[["ROUTENAME", "NDCPACKAGECODE"]].groupby(["ROUTENAME"]).count()\
.reset_index()\
.rename(columns= {"NDCPACKAGECODE" : "Count of Drugs", "ROUTENAME" : "Route"})\
.sort_values(by= "Count of Drugs", ascending=False)\

route_df.head()

,Route,Count of Drugs
167,oral,138948
199,topical,51680
105,intradermal; subcutaneous,12674
189,respiratory (inhalation),9179
142,intravenous,4352


In [71]:
marketing_df = ndc_df[["MARKETINGCATEGORYNAME", "NDCPACKAGECODE"]].groupby(["MARKETINGCATEGORYNAME"]).count()\
.reset_index()\
.rename(columns= {"NDCPACKAGECODE" : "Count of Drugs", "MARKETINGCATEGORYNAME" : "Marketing Category"})\
.sort_values(by= "Count of Drugs", ascending=False)\

marketing_df.head()

,Marketing Category,Count of Drugs
0,anda,99710
5,otc monograph not final,35831
4,otc monograph final,29673
8,unapproved homeopathic,22962
1,bla,19917


In [72]:
labeler_df = ndc_df[["LABELERNAME", "NDCPACKAGECODE"]].groupby(["LABELERNAME"]).count()\
.reset_index()\
.rename(columns= {"NDCPACKAGECODE" : "Count of Drugs", "LABELERNAME" : "Labeler"})\
.sort_values(by= "Count of Drugs", ascending=False)\

labeler_df.head()

,Labeler,Count of Drugs
5315,"nelco laboratories, inc.",9448
1310,bryant ranch prepack,9386
8170,washington homeopathic products,5982
6106,preferred pharmaceuticals,5186
6655,"rxhomeo private limited d.b.a. rxhomeo, inc",4656


In [73]:
marketingyear_df = ndc_df[["Marketing Start Year", "NDCPACKAGECODE"]].groupby(["Marketing Start Year"]).count()\
.reset_index()\
.rename(columns= {"NDCPACKAGECODE" : "Count of Drugs"})\
.sort_values(by= "Marketing Start Year", ascending=True)\

marketingyear_df.head()

,Marketing Start Year,Count of Drugs
0,1900,48
1,1905,1
2,1907,1
3,1919,29
4,1920,17


## Write DataFrames to Excel

In [77]:
wb = Workbook()
coverpage = wb.active
coverpage.sheet_properties.tabColor = "1072BA"
coverpage["A1"] = "National Drug Code Directory : Summary Tables"

coverpage["A3"] = "For more information, visit https://www.fda.gov/Drugs/InformationOnDrugs/ucm142438.htm"

coverpage["A5"] = "Drug questions email: DRUGINFO@FDA.HHS.GOV"

coverpage["A10"] = "Directory Managed By:"
coverpage["A11"] = "U.S Department of Health and Human Services" 
coverpage["A12"] = "Public Health Service" 
coverpage["A13"] = "Food and Drug Administration" 
coverpage["A14"] = "Center for Drug Evaluation and Research" 
coverpage["A15"] = "Division of Data Management and Services"

coverpage.title = "NDC Summary"


drugtype_summary = wb.create_sheet("# Drugs by DrugType")
for r in dataframe_to_rows(drugtype_df, index=False):
    drugtype_summary.append(r)
for cell in drugtype_summary['A'] + drugtype_summary[1]:
    cell.style = 'Pandas'
drugtype_summary.title = "# Drugs by DrugType"

dosage_summary = wb.create_sheet("# Drugs by Dosage")
for r in dataframe_to_rows(dosage_df, index=False):
    dosage_summary.append(r)
for cell in dosage_summary['A'] + dosage_summary[1]:
    cell.style = 'Pandas'
dosage_summary.title = "# Drugs by Dosage"

route_summary = wb.create_sheet("# Drugs by Route")
for r in dataframe_to_rows(route_df, index=False):
    route_summary.append(r)
for cell in route_summary['A'] + route_summary[1]:
    cell.style = 'Pandas'
route_summary.title = "# Drugs by Route"

marketing_summary = wb.create_sheet("# Drugs by MarketingCat")
for r in dataframe_to_rows(marketing_df, index=False):
    marketing_summary.append(r)
for cell in marketing_summary['A'] + marketing_summary[1]:
    cell.style = 'Pandas'
marketing_summary.title = "# Drugs by MarketingCategory"

labeler_summary = wb.create_sheet("# Drugs by Labeler")
for r in dataframe_to_rows(labeler_df, index=False):
    labeler_summary.append(r)
for cell in labeler_summary['A'] + labeler_summary[1]:
    cell.style = 'Pandas'
labeler_summary.title = "# Drugs by Labeler"

marketingyear_summary = wb.create_sheet("# Drugs by MarketingYear")
for r in dataframe_to_rows(marketingyear_df, index=False):
    marketingyear_summary.append(r)
for cell in marketingyear_summary['A'] + marketingyear_summary[1]:
    cell.style = 'Pandas'
marketingyear_summary.title = "# Drugs by MarketingYear"


wb.save('./output/NDC_Summary_Tables.xlsx')